# **MUSIC RECOMMENDER SYSTEM USING PYSPARK** 

### Yu chien (Calvin) Ma

Installing pyspark and other dependencies

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# !conda install pyspark
# !conda install openjdk
# !conda install -c conda-forge findspark

Importing the modules

In [1]:
import findspark
findspark.init()
findspark.find()

'C:\\Users\\calvi\\anaconda3\\lib\\site-packages\\pyspark'

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, desc , col, max
from pyspark.ml.feature import  StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder

Creating the spark session


In [3]:
# spark = SparkSession.builder.appName('rec').getOrCreate()

spark = SparkSession.builder \
.appName('app_name') \
.master('local[*]') \
.config('spark.sql.execution.arrow.pyspark.enabled', True) \
.config('spark.sql.session.timeZone', 'UTC') \
.config('spark.driver.memory','16G') \
.config('spark.ui.showConsoleProgress', True) \
.config('spark.sql.repl.eagerEval.enabled', True) \
.getOrCreate()

# Loading the dataset

In [4]:
filepath = r"C:\Users\calvi\Documents\Portfolio Projects\Music_Recommender_System_Dataset\listenings.csv"
df_listenings = spark.read.format('csv').option('header', True).option('InferSchema', True).load(filepath)
# When True, Spark will automatically infer the schema of each column


In [5]:
df_listenings.show()

+-----------+-------------+--------------------+---------------+--------------------+
|    user_id|         date|               track|         artist|               album|
+-----------+-------------+--------------------+---------------+--------------------+
|000Silenced|1299680100000|           Price Tag|       Jessie J|         Who You Are|
|000Silenced|1299679920000|Price Tag (Acoust...|       Jessie J|           Price Tag|
|000Silenced|1299679440000|Be Mine! (Ballad ...|          Robyn|            Be Mine!|
|000Silenced|1299679200000|            Acapella|          Kelis|            Acapella|
|000Silenced|1299675660000|   I'm Not Invisible|      The Tease|   I'm Not Invisible|
|000Silenced|1297511400000|Bounce (Feat NORE...|       MSTRKRFT|         Fist of God|
|000Silenced|1294498440000|Don't Stop The Mu...|        Rihanna|Addicted 2 Bassli...|
|000Silenced|1292438340000|               ObZen|      Meshuggah|               ObZen|
|000Silenced|1292437740000|   Yama's Messengers|      


# Data cleaning 

In [6]:
df_listenings = df_listenings.drop('date')
df_listenings.show()

+-----------+--------------------+---------------+--------------------+
|    user_id|               track|         artist|               album|
+-----------+--------------------+---------------+--------------------+
|000Silenced|           Price Tag|       Jessie J|         Who You Are|
|000Silenced|Price Tag (Acoust...|       Jessie J|           Price Tag|
|000Silenced|Be Mine! (Ballad ...|          Robyn|            Be Mine!|
|000Silenced|            Acapella|          Kelis|            Acapella|
|000Silenced|   I'm Not Invisible|      The Tease|   I'm Not Invisible|
|000Silenced|Bounce (Feat NORE...|       MSTRKRFT|         Fist of God|
|000Silenced|Don't Stop The Mu...|        Rihanna|Addicted 2 Bassli...|
|000Silenced|               ObZen|      Meshuggah|               ObZen|
|000Silenced|   Yama's Messengers|         Gojira|The Way of All Flesh|
|000Silenced|On the Brink of E...|   Napalm Death|Time Waits For No...|
|000Silenced|On the Brink of E...|   Napalm Death|Time Waits For

In [7]:
df_listenings = df_listenings.na.drop()

In [8]:
df_listenings.show()

+-----------+--------------------+---------------+--------------------+
|    user_id|               track|         artist|               album|
+-----------+--------------------+---------------+--------------------+
|000Silenced|           Price Tag|       Jessie J|         Who You Are|
|000Silenced|Price Tag (Acoust...|       Jessie J|           Price Tag|
|000Silenced|Be Mine! (Ballad ...|          Robyn|            Be Mine!|
|000Silenced|            Acapella|          Kelis|            Acapella|
|000Silenced|   I'm Not Invisible|      The Tease|   I'm Not Invisible|
|000Silenced|Bounce (Feat NORE...|       MSTRKRFT|         Fist of God|
|000Silenced|Don't Stop The Mu...|        Rihanna|Addicted 2 Bassli...|
|000Silenced|               ObZen|      Meshuggah|               ObZen|
|000Silenced|   Yama's Messengers|         Gojira|The Way of All Flesh|
|000Silenced|On the Brink of E...|   Napalm Death|Time Waits For No...|
|000Silenced|On the Brink of E...|   Napalm Death|Time Waits For

In [9]:
rows = df_listenings.count()
cols = len(df_listenings.columns)
print ("Rows: ",rows, " and Columns: ", cols)

Rows:  13758905  and Columns:  4



# Performing aggregation
(Seeing how many times each user has listened to specific track)


In [10]:
df_listenings_agg = df_listenings.select('user_id','track').groupby('user_id',
                    'track').agg(count('*').alias('count')).orderBy('user_id')
df_listenings_agg.show()

+-------+--------------------+-----+
|user_id|               track|count|
+-------+--------------------+-----+
| --Seph|In the Nothing of...|    2|
| --Seph|Chelsea Hotel - L...|    1|
| --Seph|        Window Blues|    1|
| --Seph|     The Way We Were|    1|
| --Seph|Hungarian Rhapsod...|    1|
| --Seph|Belina (Original ...|    1|
| --Seph|          Paris 2004|    7|
| --Seph|              Monday|    1|
| --Seph|Vestido Estampado...|    1|
| --Seph|   Summa for Strings|    1|
| --Seph|      Hour for magic|    2|
| --Seph|       Phantom Pt II|    1|
| --Seph|       Life On Mars?|    1|
| --Seph|Hungarian Dance No 5|    1|
| --Seph|         The Embrace|    1|
| --Seph|  California Waiting|    1|
| --Seph|Airplanes [feat H...|    1|
| --Seph|Virus (Luke Fair ...|    1|
| --Seph| Air on the G String|    1|
| --Seph| White Winter Hymnal|    3|
+-------+--------------------+-----+
only showing top 20 rows



In [11]:
rows = df_listenings_agg.count()
cols = len(df_listenings_agg.columns)
print ("Rows: ",rows, " and Columns: ", cols)

Rows:  9930128  and Columns:  3


In [12]:
df_listenings_agg = df_listenings_agg.limit(20000)

# Converting the user id and track columns into unique integers




In [13]:
indexer = [StringIndexer(inputCol=col,outputCol=col+"_Index").fit(df_listenings_agg) for col in list(
    set(df_listenings_agg.columns)-set('count'))]

In [14]:
pipeline = Pipeline(stages=indexer)

In [15]:
data = pipeline.fit(df_listenings_agg).transform(df_listenings_agg)

In [16]:
data.show()

+-------+--------------------+-----+-----------+-------------+-----------+
|user_id|               track|count|track_Index|user_id_Index|count_Index|
+-------+--------------------+-----+-----------+-------------+-----------+
| --Seph| White Winter Hymnal|    3|       59.0|         69.0|        2.0|
| --Seph|Virus (Luke Fair ...|    1|    15896.0|         69.0|        0.0|
| --Seph|Airplanes [feat H...|    1|      519.0|         69.0|        0.0|
| --Seph|Belina (Original ...|    1|     3278.0|         69.0|        0.0|
| --Seph|              Monday|    1|      334.0|         69.0|        0.0|
| --Seph|Hungarian Dance No 5|    1|     7558.0|         69.0|        0.0|
| --Seph|       Life On Mars?|    1|     1161.0|         69.0|        0.0|
| --Seph|  California Waiting|    1|      197.0|         69.0|        0.0|
| --Seph|       Phantom Pt II|    1|     1377.0|         69.0|        0.0|
| --Seph|   Summa for Strings|    1|    13739.0|         69.0|        0.0|
| --Seph|      Hour for m

# Train and test data

In [29]:
# training, testing = data.limit(100).randomSplit([0.7, 0.3])
(training, testing) = data.limit(1000).randomSplit([0.7, 0.3])


In [30]:
training.show(5)

+-------+------------+-----+-----------+-------------+-----------+
|user_id|       track|count|track_Index|user_id_Index|count_Index|
+-------+------------+-----+-----------+-------------+-----------+
| --Seph|        0040|    1|     1924.0|         69.0|        0.0|
| --Seph|     401 Lwa|    2|     2076.0|         69.0|        1.0|
| --Seph|5 Years Time|    1|     2090.0|         69.0|        0.0|
| --Seph|After Nature|    1|      513.0|         69.0|        0.0|
| --Seph| Agoraphobia|    1|     2426.0|         69.0|        0.0|
+-------+------------+-----+-----------+-------------+-----------+
only showing top 5 rows



# Creating the model

In [31]:
USERID = 'user_id_Index'
TRACK = 'track_Index'
COUNT = 'count'

als= ALS(maxIter=5, regParam=0.01, userCol=USERID, itemCol=TRACK, ratingCol=COUNT)



In [32]:
# spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

In [33]:
# ALS requires that the column IDs be numeric instead of strings
model= als.fit(training)

In [34]:
predictions = model.transform(testing)


# Generating the top 10 track recommendations for each user

In [35]:
recs = model.recommendForAllUsers(10)

In [36]:
recs.show()

+-------------+--------------------+
|user_id_Index|     recommendations|
+-------------+--------------------+
|          122|[{235, 16.996416}...|
|            1|[{4713, 1.9809115...|
|           92|[{16909, 5.295038...|
|          107|[{16909, 29.98323...|
|           84|[{5937, 8.97173},...|
|           69|[{11313, 6.966994...|
|           10|[{6972, 1.9843587...|
|          110|[{13563, 12.98881...|
+-------------+--------------------+



In [37]:
recs.take(1)

[Row(user_id_Index=122, recommendations=[Row(track_Index=235, rating=16.996416091918945), Row(track_Index=15793, rating=16.996416091918945), Row(track_Index=12061, rating=16.996416091918945), Row(track_Index=14538, rating=14.996837615966797), Row(track_Index=4995, rating=14.996837615966797), Row(track_Index=8705, rating=14.996837615966797), Row(track_Index=6515, rating=14.996837615966797), Row(track_Index=4220, rating=14.996837615966797), Row(track_Index=12845, rating=14.996837615966797), Row(track_Index=14799, rating=12.997260093688965)])]